In [1]:
%env KITTI_ROOT=/data/kitti_lidar
!export KITTI_ROOT=/data/kitti_lidar

env: KITTI_ROOT=/data/kitti_lidar


In [2]:
%env NUMBA_WARNINGS=0

env: NUMBA_WARNINGS=0


### To use different copy of `second.pytorch`

In [3]:
# %%bash
# cd /home/pytorch/external/ && git clone https://github.com/jitrc/second.pytorch -b quick_fix
# export SECOND_API=/home/pytorch/external/second.pytorch/second
# export PYTHONPATH=/home/pytorch/external/second.pytorch:${PYTHONPATH}

In [4]:
# %env SECOND_API=/home/pytorch/external/second.pytorch/second
# %env PYTHONPATH=/home/pytorch/external/second.pytorch:${PYTHONPATH}

In [5]:
!ls ${SECOND_API}

__init__.py  create_data.py  protos	       simple-inference.ipynb
builder      data	     pytorch	       utils
configs      framework	     script.py
core	     kittiviewer     script_server.py


In [6]:
!ls ${KITTI_ROOT}

testing  training


In [7]:
!tree --filelimit 10 ${KITTI_ROOT}

/data/kitti_lidar
|-- testing
|   |-- calib [7518 entries exceeds filelimit, not opening dir]
|   |-- image_2 [7518 entries exceeds filelimit, not opening dir]
|   `-- velodyne [7518 entries exceeds filelimit, not opening dir]
`-- training
    |-- calib [7481 entries exceeds filelimit, not opening dir]
    |-- image_2 [7481 entries exceeds filelimit, not opening dir]
    |-- label_2 [7481 entries exceeds filelimit, not opening dir]
    `-- velodyne [7481 entries exceeds filelimit, not opening dir]

9 directories, 0 files


# Prepare dataset

In [8]:
!mkdir -p ${KITTI_ROOT}/training/velodyne_reduced/
!mkdir -p ${KITTI_ROOT}/testing/velodyne_reduced/

In [9]:
# !python ${SECOND_API}/create_data.py kitti_data_prep ${KITTI_ROOT}

Generate info. this may take several minutes.
Kitti info train file is saved to /data/kitti_lidar/kitti_infos_train.pkl
Kitti info val file is saved to /data/kitti_lidar/kitti_infos_val.pkl
Kitti info trainval file is saved to /data/kitti_lidar/kitti_infos_trainval.pkl
Kitti info test file is saved to /data/kitti_lidar/kitti_infos_test.pkl
[100.0%][===================>][184.77it/s][00:19>00:00]   
[100.0%][===================>][195.29it/s][00:21>00:00]   
[100.0%][===================>][189.51it/s][00:40>00:00]   
remain number of infos: 3712
[100.0%][===================>][550.14it/s][00:06>00:00]   
load 2207 Pedestrian database infos
load 14357 Car database infos
load 734 Cyclist database infos
load 1297 Van database infos
load 488 Truck database infos
load 224 Tram database infos
load 337 Misc database infos
load 56 Person_sitting database infos


In [10]:
!ls -l --block-size=M /data/kitti_lidar/*.pkl

-rw-r--r-- 1 jit jit  6M Jun 24 00:02 /data/kitti_lidar/kitti_dbinfos_train.pkl
-rw-r--r-- 1 jit jit 11M Jun 24 00:00 /data/kitti_lidar/kitti_infos_test.pkl
-rw-r--r-- 1 jit jit 13M Jun 23 23:59 /data/kitti_lidar/kitti_infos_train.pkl
-rw-r--r-- 1 jit jit 25M Jun 24 00:00 /data/kitti_lidar/kitti_infos_trainval.pkl
-rw-r--r-- 1 jit jit 13M Jun 24 00:00 /data/kitti_lidar/kitti_infos_val.pkl


# Configure

In [11]:
!ls -R ${SECOND_API}/configs

/opt/second.pytorch/second/configs:
all.fhd.config	car.fhd.onestage.config  nuscenes	    pointpillars
car.fhd.config	car.lite.config		 people.fhd.config

/opt/second.pytorch/second/configs/nuscenes:
all.fhd.config		  all.pp.largea.config	all.pp.mhead.config
all.pp.deprecated.config  all.pp.lowa.config	all.pp.mida.config

/opt/second.pytorch/second/configs/pointpillars:
car  ped_cycle	pp_pretrain.config

/opt/second.pytorch/second/configs/pointpillars/car:
xyres_16.config  xyres_20.config  xyres_24.config  xyres_28.config

/opt/second.pytorch/second/configs/pointpillars/ped_cycle:
xyres_16.config  xyres_20.config  xyres_24.config  xyres_28.config


In [12]:
%env KITTI_ROOT=/data/kitti_lidar
%env MODEL_OUT_PATH=/data/kitti_lidar/trained_model
%env MODEL_NAME=car.lite
%env MODEL_NAME_FLAT=car_lite

env: KITTI_ROOT=/data/kitti_lidar
env: MODEL_OUT_PATH=/data/kitti_lidar/trained_model
env: MODEL_NAME=car.lite
env: MODEL_NAME_FLAT=car_lite


In [13]:
%%bash
# Use this block to copy/paste in terminal 

# export KITTI_ROOT='/data/kitti_lidar'
# export MODEL_OUT_PATH="${KITTI_ROOT}/trained_model"
# export MODEL_NAME='car.fhd' # 'all.fhd', 'car.fhd', `pointpillars/cars/xyres_16` ...
# export MODEL_NAME_FLAT=$(echo ${MODEL_NAME//\//_}| tr "." _)

In [14]:
!echo $MODEL_NAME_FLAT

car_lite


## Update config file

In [15]:
%%bash
mkdir -p ${MODEL_OUT_PATH}
cp ${SECOND_API}/configs/${MODEL_NAME}.config ${MODEL_OUT_PATH}/train.config

sed -i "s|kitti_root_path:.*|kitti_root_path: \"${KITTI_ROOT}\"|g" ${MODEL_OUT_PATH}/train.config
sed -i "s|kitti_info_path:.*/kitti_infos_train.pkl\"|kitti_info_path: \"${KITTI_ROOT}/kitti_infos_train.pkl\"|g" ${MODEL_OUT_PATH}/train.config
sed -i "s|kitti_info_path:.*/kitti_infos_val.pkl\"|kitti_info_path: \"${KITTI_ROOT}/kitti_infos_val.pkl\"|g" ${MODEL_OUT_PATH}/train.config
sed -i "s|kitti_info_path:.*/kitti_infos_test.pkl\"|kitti_info_path: \"${KITTI_ROOT}/kitti_infos_test.pkl\"|g" ${MODEL_OUT_PATH}/train.config
sed -i "s|database_info_path:.*|database_info_path: \"${KITTI_ROOT}/kitti_dbinfos_train.pkl\"|g" ${MODEL_OUT_PATH}/train.config

# sed -i "s|batch_size:.*|batch_size: 6|g" ${MODEL_OUT_PATH}/train.config
# sed -i "s|num_workers:.*|num_workers: 4|g" ${MODEL_OUT_PATH}/train.config

cat ${MODEL_OUT_PATH}/train.config

model: {
  second: {
    network_class_name: "VoxelNet"
    voxel_generator {
      # point_cloud_range : [0, -40, -3, 70.4, 40, 1]
      point_cloud_range : [0, -32.0, -3, 52.8, 32.0, 1]
      # point_cloud_range : [-50, -50.0, -3, 50, 50, 1]
      voxel_size : [0.05, 0.05, 0.1]
      max_number_of_points_per_voxel : 1
      block_filtering: false # filter voxels by block height
      block_factor: 1 # block size: voxel_size * block_factor * block_size = 0.05 * 1 * 8 = 0.4
      block_size: 8
      height_threshold: 0.2
    }

    voxel_feature_extractor: {
      module_class_name: "SimpleVoxelRadius"
      num_filters: [16]
      with_distance: false
      num_input_features: 4
    }
    middle_feature_extractor: {
      module_class_name: "SpMiddleFHDLite"
      # num_filters_down1: [] # protobuf don't support empty list.
      # num_filters_down2: []
      downsample_factor: 8
      num_input_features: 3
    }
    rpn: {
      module_class_name: "RPNV2"
      layer_nums: [5]
      

# Evalute Pretrained

In [16]:
!ls /data/pretrained_models/Pointpillars_models_v1.5

car_fhd  car_lite  car_onestage  pp_model_for_nuscenes_pretrain


In [17]:
!python ${SECOND_API}/pytorch/train.py evaluate --config_path=${MODEL_OUT_PATH}/train.config --model_dir=/data/pretrained_models/Pointpillars_models_v1.5/${MODEL_NAME_FLAT}

[  41 1280 1056]
Restoring parameters from /data/pretrained_models/Pointpillars_models_v1.5/car_lite/voxelnet-15500.tckpt
feature_map_size [1, 160, 132]
remain number of infos: 3769
Generate output labels...
[100.0%][===================>][12.27it/s][00:26>00:00]   
generate label finished(140.19/s). start eval:
/usr/local/lib/python3.6/dist-packages/numba/core/typed_passes.py:314: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../opt/second.pytorch/second/utils/eval.py", line 129:
@numba.jit(nopython=True, parallel=True)
def box3d_overlap_kernel(boxes,
^

  state.func_ir.loc))
Evaluation official
Car AP(Average Precision)@0.70, 0.70, 0.70:
bbox AP:90.64, 89.27, 87.86
bev  AP:90.06, 87.28, 86.39
3d   AP:88.06, 77.69, 75.41
aos  AP:90.55, 88.69, 87.04


# Training

## Tensorboard

In [18]:
%load_ext tensorboard.notebook
%tensorboard --logdir /data/kitti_lidar/trained_model

##  Prepate output directory

In [19]:
!echo ${MODEL_OUT_PATH}/${MODEL_NAME_FLAT}
!rm -rf ${MODEL_OUT_PATH}/${MODEL_NAME_FLAT}

/data/kitti_lidar/trained_model/car_lite


## Train with single GPU

In [20]:
!python ${SECOND_API}/pytorch/train.py train --config_path=${MODEL_OUT_PATH}/train.config --model_dir=${MODEL_OUT_PATH}/${MODEL_NAME_FLAT}

[  41 1280 1056]
num parameters: 39
False _amp_stash
{'Car': 5}
[-1]
load 2207 Pedestrian database infos
load 14357 Car database infos
load 734 Cyclist database infos
load 1297 Van database infos
load 488 Truck database infos
load 224 Tram database infos
load 337 Misc database infos
load 56 Person_sitting database infos
After filter database:
load 2207 Pedestrian database infos
load 13442 Car database infos
load 734 Cyclist database infos
load 1297 Van database infos
load 488 Truck database infos
load 224 Tram database infos
load 337 Misc database infos
load 56 Person_sitting database infos
feature_map_size [1, 160, 132]
remain number of infos: 3712
feature_map_size [1, 160, 132]
remain number of infos: 3769
model: {
  second: {
    network_class_name: "VoxelNet"
    voxel_generator {
      # point_cloud_range : [0, -40, -3, 70.4, 40, 1]
      point_cloud_range : [0, -32.0, -3, 52.8, 32.0, 1]
      # point_cloud_range : [-50, -50.0, -3, 50, 50, 1]
      voxel_size : [0.05, 0.05, 0.1]
 

## Train with Multi GPU
not tested

In [21]:
# !CUDA_VISIBLE_DEVICES=0,1 python ${SECOND_API}/pytorch/train.py train --config_path=${MODEL_OUT_PATH}/train.config --model_dir=${MODEL_OUT_PATH}/${MODEL_NAME_FLAT} --multi_gpu=True

# Evaluate

In [22]:
!python ${SECOND_API}/pytorch/train.py evaluate --config_path=${MODEL_OUT_PATH}/train.config --model_dir=${MODEL_OUT_PATH}/${MODEL_NAME_FLAT}

[  41 1280 1056]
Restoring parameters from /data/kitti_lidar/trained_model/car_lite/voxelnet-15500.tckpt
feature_map_size [1, 160, 132]
remain number of infos: 3769
Generate output labels...
[100.0%][===================>][12.30it/s][00:26>00:00]   
generate label finished(141.54/s). start eval:
/usr/local/lib/python3.6/dist-packages/numba/core/typed_passes.py:314: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../opt/second.pytorch/second/utils/eval.py", line 129:
@numba.jit(nopython=True, parallel=True)
def box3d_overlap_kernel(boxes,
^

  state.func_ir.loc))
Evaluation official
Car AP(Average Precision)@0.70, 0.70, 0.70:
bbox AP:90.62, 88.96, 87.32
bev  AP:89.85, 86.12, 86.10
3d   AP:86.92, 77.14, 74.97
aos  AP:0.64, 1.53, 2.22
Car AP(Average Preci